In [1]:
import os
import pandas as pd
import requests
import json
import re
from tqdm import tqdm
import time
import numpy as np
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import datetime as dt
from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook

pd.set_option('display.max_colwidth', None)

In [2]:
today_date = dt.date.today().strftime('%Y_%m_%d')

In [3]:
desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
database_path = desktop_path+'/Databases'
git_path = desktop_path +'/Beacon_Internal/Projects/CDM_Smith/2021_10/Code/Source'

git_path

'/Users/beaconeconomics/Desktop/Beacon_Internal/Projects/CDM_Smith/2021_10/Code/Source'

# 1. RIVCOM_TAZ Shapefile

In [4]:
df_riv = pd.read_csv(git_path+'/RIVCOM_TAZ_2021_10_20.csv')

df_riv.head(2)

,Unnamed: 0,ID,AREA,ACRES,COUNTY,CITY,TIER2_SCAG,TAZ
0,0,1,0.402481,257,Riverside,BANNING,43462100,1
1,1,2,0.101948,65,Riverside,BANNING,43462100,2


In [5]:
set(df_riv.CITY)

{'BANNING',
 'BEAUMONT',
 'BLYTHE',
 'CALIMESA',
 'CANYON LAKE',
 'CATHEDRAL CITY',
 'COACHELLA',
 'CORONA',
 'DESERT HOT SPRINGS',
 'EASTVALE',
 'HEMET',
 'INDIAN WELLS',
 'INDIO',
 'JURUPA VALLEY',
 'LA QUINTA',
 'LAKE ELSINORE',
 'MENIFEE',
 'MORENO VALLEY',
 'MURRIETA',
 'NORCO',
 'PALM DESERT',
 'PALM SPRINGS',
 'PERRIS',
 'RANCHO MIRAGE',
 'RIVERSIDE',
 'SAN JACINTO',
 'TEMECULA',
 'Unincorporated/Not a city',
 'WILDOMAR',
 nan}

In [6]:
df_riv.CITY=df_riv.CITY.str.title()

In [7]:
shp_city= {x for x in df_riv.CITY if pd.notna(x)}
shp_city.remove('Unincorporated/Not A City')
len(shp_city)

28

In [8]:
target_city = ['Murrieta','Wildomar','Lake Elsinore','Corona','Norco','EastVale']

# 3. CIRB

In [9]:
desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
database_path = desktop_path+'/Databases'

# NOTE CIRB data contains cities.dta as well as county.dta
df_cirb = pd.read_stata(database_path+'/California/CIRB/Cities.dta')
df_cirb.head(2)

,place,county,variable,quarter,value,value_sa,value_nsa,units,description
0,Adelanto,San Bernardino,MULTIUNIT,1996-01-01,0.0,0.0,0,Number of Permits,Multi-Family Units
1,Adelanto,San Bernardino,MULTIUNIT,1996-04-01,0.0,0.0,0,Number of Permits,Multi-Family Units


# 3.1 Pull Data 

In [10]:
def pull_cirb(df,location):
    
    df_re = df
    
    cats = ['VCOMM','VHOTEL','VNRALTS','VOFFICE','VINDL','VSTORE','MULTIUNIT','SFUNITS','TOTNEW','VNRTOTAL']

    mask = df_re.variable.isin(cats)& df_re.place.isin(location)
    df_re = df_re.loc[mask]

    df_re = df_re[['place','quarter','variable','value_sa','units']]

   
    df_re = df_re.pivot_table(index= ['place','quarter'], columns=['variable'], values= 'value_sa')
    
   
    return df_re

In [11]:
df_cirb = pull_cirb(df_cirb,target_city)
df_cirb.head(2)

variable           MULTIUNIT     SFUNITS      TOTNEW      VCOMM  VHOTEL  \
place  quarter                                                            
Corona 1996-01-01        0.0  476.596569  495.325547   170000.0     0.0   
       1996-04-01        0.0  399.560091  425.109708  3067507.0     0.0   

variable               VINDL       VNRALTS      VNRTOTAL  VOFFICE     VSTORE  
place  quarter                                                                
Corona 1996-01-01  4955664.0  4.500001e+06  9.202702e+06      0.0   170000.0  
       1996-04-01  1416662.0  1.310875e+06  7.790431e+06      0.0  3067507.0

# 2. ACS At CITY LEVEL

In [12]:
class api_fetch_city:   
    def __init__(self, variable_name, entity, database, year):
        
        self.year = year
        self.base = f'https://api.census.gov/data/{self.year}/acs/'
        self.api_base = self.base+database
        self.api_key = 'key=bc362b9bc359b0b06cc7b4825f843315585beffc'
        self.variable_name = "?&get=NAME,"+variable_name + entity
        
        self.api_final = f'{self.api_base}{self.variable_name}{self.api_key}'
        
    def request(self):
        return requests.get(self.api_final)
    
    def download(self):
        r = requests.get(self.api_final)
        data = r.json()
        df = pd.DataFrame(data)
        df = df.rename(columns=df.iloc[0]).drop(df.index[0])
        df = pd.concat([df.NAME, df.loc[:, ~df.columns.isin(['NAME'])].astype(float)],axis=1)
        return df
  

In [13]:
codes = {'B25008_003E': 'pop_renter_check',
 'B06011_001E': 'median_inc',
 'B01003_001E': 'pop_total',
 'B01002_001E': 'age',
 'B02001_002E': 'pop_white',
 'B02001_003E': 'pop_black',
 'B25106_028E': 'Burden_30 Less than 20,000',
 'B25106_032E': 'Burden_30 $20,000 to $34,999:',
 'B25106_036E': 'Burden_30 $35,000 to $49,999:',
 'B25106_040E': 'Burden_30 $50,000 to $74,999:',
 'B25106_044E': 'Burden_30 $75,000 or more:',
 'B25111_001E': 'Med_rent'}

In [14]:
city = '&for=place:*&'
df = api_fetch_city('B25008_003E', city, 'acs5', str(2019)).download()
mask = df.state == 6
df = df.loc[mask, :]
df.head(2)

,NAME,B25008_003E,state,place
359,"Moss Landing CDP, California",0.0,6.0,49488.0
360,"Belvedere city, California",517.0,6.0,5164.0


In [15]:
df_merge = []

start_year = 2010
for code, series in tqdm(codes.items()):
    data = []
    for year in range(start_year, 2020):
        df = api_fetch_city(code, city, 'acs5', str(year)).download()
        mask = df.state==6
        df = df.loc[mask].drop(['state','place'],axis=1).set_index('NAME')
        df.columns = [str(year)]
        data.append(df)
    
    df_counties = pd.concat(data, axis=1)
    df_counties = df_counties.transpose()
    df_counties = df_counties.reset_index().melt(id_vars = 'index', value_name = series).set_index(['index','variable'])

    #df_counties = df_counties.set_index(['location','time'])
    df_merge.append(df_counties)

100%|██████████| 12/12 [07:11<00:00, 35.93s/it]


In [16]:
city_final = pd.concat(df_merge,axis=1)
city_final.reset_index(inplace=True)


In [17]:
city_final[['city', 'state']] = city_final.variable.str.split(',',expand=True)

In [18]:
regex_pat = re.compile(r'CDP|city', flags=re.IGNORECASE)
city_final.city=city_final.city.str.replace(regex_pat, '', regex=True)

In [19]:
city_final.city = city_final.city.apply(lambda x: x.rstrip())


# Count of Cities from shapefile in each data source

In [20]:
set2= set(city_final.city)

In [21]:
acs_shp=(shp_city.intersection(set2))
len(acs_shp)

27

In [22]:
set3 = set(df_cirb.place)

AttributeError: 'DataFrame' object has no attribute 'place'

In [ ]:
cirb_shp=(shp_city.intersection(set3))
len(cirb_shp)

In [ ]:
print(set3&shp_city)

In [ ]:
set4 = set(df_hp.city)

In [ ]:
dq_shp=(shp_city.intersection(set4))
len(dq_shp)

In [ ]:
d1=dict(enumerate(cirb_city))
d1

In [ ]:
#city_final = city_final.loc[:, ~city_final.columns.duplicated()]

In [ ]:
#mask=city_final.city.isin(cirb_city)
#city_final= city_final.loc[mask,:]
#city_final

# 2.1.1 ACS Final Format for mergeing 

In [ ]:
city_final.rename(columns={'index':'year'},inplace=True)
city_final.year = pd.PeriodIndex(pd.to_datetime(city_final.year),freq='A')
mask= (city_final.city.isin(city_list))
city_final= city_final.loc[mask,:]
city_final.set_index(['city','year'],inplace=True)
city_final.drop(columns=['state','variable'],inplace=True)
city_final

In [ ]:
city_final = city_final.pivot_table(index= ['city','year'])

In [ ]:
city_final.index = pd.MultiIndex.from_arrays([
    city_final.index.get_level_values(0),
    pd.PeriodIndex(city_final.index.get_level_values(1), freq='A')])

## 2.2 Convert to annual

In [ ]:
city_final

In [ ]:
manip = {'MULTIUNIT': 'sum',
                    'SFUNITS': 'sum',
                    'VCOMM': 'mean',
                    'VHOTEL': 'mean',
                    'VINDL': 'mean',
                    'VNRALTS': 'mean',
                    'VOFFICE': 'mean',
                    'VSTORE': 'mean',
                    'TOTNEW': 'sum',
                    'VNRTOTAL': 'sum'}

In [ ]:
df_cirb = df_cirb.reset_index().set_index('quarter').groupby('place').resample('A').agg(manip)

# 4. Merge CIRB / ACS

## 4.1 Check county match

In [ ]:
df_cirb.index.get_level_values(0).unique()

In [ ]:
city_final.index.get_level_values(0).unique()

# 4.1.1 Double check list comprehension 

In [ ]:
[x for x in df_cirb.index.get_level_values(0).unique() if x in city_final.index.get_level_values(0).unique()]

## 4.2 Check time series match

In [ ]:
df_cirb.index.get_level_values(1).unique()

In [ ]:
df_cirb.index

In [ ]:
city_final.index.get_level_values(1).unique()

# 4.2.2 Convert to PeriodIndex 

In [ ]:
df_cirb.index = pd.MultiIndex.from_arrays([
    df_cirb.index.get_level_values(0),
    pd.PeriodIndex(df_cirb.index.get_level_values(1), freq='A')])
df_cirb.index[:3]

# 4.3 CIRB/ ACS Merge

## Axis =1 

In [ ]:
frames = [city_final,df_cirb]  # Or perform operations on the DFs
result = pd.concat(frames,axis=1)

In [ ]:
result[15:24]

# 5. DataQuick

In [ ]:
df_hp = pd.read_stata(database_path+"/California/DataQuick/City.dta")
df_hp.head(3)

In [ ]:
df_hp.variable.unique()

In [ ]:
def pull_dq(df, location):
    
    df_re = df
    
    mask =  (df_re.city.isin(location))
    df_re = df_re.loc[mask]
   
    df_re = df_re[['city','quarter','value','variable']] 
   
    return df_re

In [ ]:
dq_data=pull_dq(df_hp, city_list)
dq_data

# 5. Merge CIRB / ACS

In [ ]:
## 5.1 Check county match

In [ ]:
dq_data.index.get_level_values(0).unique()

In [ ]:
df_cirb.index.get_level_values(0).unique()

## 4.2 Check time series match

In [ ]:
df_cirb.index.get_level_values(1).unique()

In [ ]:
dq_data.index.get_level_values(1).unique()

### 4.2.2 Convert to period index

In [ ]:
df_cirb.index = pd.MultiIndex.from_arrays([
    df_cirb.index.get_level_values(0),
    pd.PeriodIndex(df_cirb.index.get_level_values(1), freq='A')])
df_cirb.index[:3]

In [ ]:
dq_data.index = pd.MultiIndex.from_arrays([
    dq_data.index.get_level_values(0),
    pd.PeriodIndex(dq_data.index.get_level_values(1), freq='A')])
dq_data.index[:3]

In [ ]:
dq_data = dq_data.set_index('quarter').groupby('city').resample('A').sum()

# 4.3 CIRB (28) / Data Quick (6) Merge

In [ ]:
df_merge_2 = pd.concat([df_cirb,dq_data],axis=1)
df_merge_2.head(150)

# 4.4 Merge CIRB(28) /ACS(27)/Data Quick(6)

In [ ]:
df_merge_final = pd.concat([df_cirb,dq_data,city_final],axis=1)
df_merge_final[16:20]

# 7. QCEW (1)

In [ ]:
df_qc = pd.read_stata(database_path+"/National/QCEW/counties.dta")
df_qc.rename(columns = {'size':'est_size'},inplace=True)

df_qc.area = [x.split(',')[0].replace('County', '').rstrip() for x in df_qc.area]

df_qc.head(2)

In [ ]:
print(set(df_qc['area']) & set(cirb_city))

In [ ]:
print(set(df_qc['area']) & shp_city)

In [ ]:
mask = df_qc['area'].isin(shp_city)
df_qc=df_qc.loc[mask,:]
df_qc

In [ ]:
mask = (df_qc.own == 'Total Covered') & (df_qc.est_size == 'All establishment sizes') 

df_qc_total = df_qc.loc[mask, ['area', 'numest','quarter','Wage','emplt']].set_index(['area','quarter'])

df_qc_total = df_qc_total.reset_index().set_index('quarter').groupby('area').resample('A').agg({'Wage':'sum', 'emplt':'mean'})

df_qc_total.head(2)

In [ ]:
df_qc_total.index = pd.MultiIndex.from_arrays([
    df_qc_total.index.get_level_values(0),
    pd.PeriodIndex(df_qc_total.index.get_level_values(1), freq='A')])

df_qc_total.head(2)

# 6.2 Check time series match

In [ ]:
df_qc_total.index.get_level_values(1).unique()

In [ ]:
df_qc_total.index.get_level_values(0).unique()

# 6.3  QCEW Merge

In [ ]:
_merge_final = pd.concat([df_cirb,dq_data,city_final,df_qc_total],axis=1)
_merge_final.head(2)

# 7 . Population

## 1990-2000 (19)

In [ ]:
df = pd.read_excel(git_path+'/DOF/E-4_90-00_Rpt.xls',
    sheet_name =1,
    header = 3,
    index_col=0,
    skipfooter=2
).iloc[2:,:] #skip first two rows
df.head(2)

In [ ]:
df = df.transpose().reset_index()
df.head(2)

In [ ]:
df_1990 = df.melt(id_vars='index').rename(columns={'index':'date'})#.set_index(['date','City'])
df_1990.head(2)

In [ ]:
df_1990['City']=df_1990['City'].str.rstrip()

In [ ]:
mask= (df_1990['City'].isin(city_list))
df_1990_f= df_1990.loc[mask,:]

In [ ]:
df_1990_f = df_1990_f.dropna()
df_1990_f

In [ ]:
 df_1990_f = df_1990_f.pivot_table(index= ['City','date'],values='value',aggfunc='last')

In [ ]:
df_1990_f.index = pd.MultiIndex.from_arrays([
    df_1990_f.index.get_level_values(0),
    pd.PeriodIndex(df_1990_f.index.get_level_values(1), freq='A')])

## 2010-2020 (19)

In [ ]:
df2 = pd.read_excel(git_path+'/DOF/E-4_2021InternetVersion.xlsx',
    sheet_name =2,
    header = 1,
    index_col=0,
    skipfooter=1
).iloc[1:712, :12]
df2.tail(4)

In [ ]:
df2 = df2.transpose().reset_index()
df2.head(2)

In [ ]:
df_2000 = df2.melt(id_vars='index').rename(columns={'index':'date',
                                             'COUNTY/CITY': 'city', 
                                            'value' : 'value_2000'})
df_2000.date = pd.to_datetime(df_2000.date)
#df_2000.set_index(['date','City'],inplace=True)
df_2000.head(2)

In [ ]:
print(len(set(df_2000['city'])& set(city_list)))

In [ ]:
mask= (df_2000['city'].isin(city_list))
df_2000_f= df_2000.loc[mask,:]

In [ ]:
df_2000_f = df_2000_f.pivot_table(index= ['city','date'])

In [ ]:
df_2000_f.index = pd.MultiIndex.from_arrays([
    df_2000_f.index.get_level_values(0),
    pd.PeriodIndex(df_2000_f.index.get_level_values(1), freq='A')])

# 7. Population Merge

In [ ]:
df_1990_f.index.get_level_values(0).unique()

In [ ]:
df_2000_f.index.get_level_values(0).unique()

In [ ]:
df_1990_f.index.get_level_values(1).unique()

In [ ]:
df_2000_f.index.get_level_values(1).unique()

In [ ]:
df_pop_city = pd.concat([df_1990_f, df_2000_f],axis=0)
df_pop_city


# 8. All data source merge (CIRB(28)/ ACS(27)/ Population(19)/ DataQuick(6)/QCEW(1)

# First Three 

In [ ]:
frames = [city_final,df_cirb,df_1990_f, df_2000_f]
final = pd.concat(frames, axis=1)
final[13:20]

# All (7X27)

In [ ]:
frames2 = [city_final,df_cirb,df_1990_f, df_2000_f,dq_data,df_qc_total]
final_f = pd.concat(frames2, axis=1)
final_f[13:20]

In [ ]:
#9

In [ ]:
path = 'Input/Model_dictionary/'
dict_files = [x for x in os.listdir(path) if x.endswith('.xlsx') and not x.startswith('~')]
path = 'Input/Model_dictionary/'
series = pd.read_excel(path + dict_files[0],sheet_name='Series')
series = series.loc[series.Series!=0]
var_list = []
for series_name, entity_name in zip(series.Series, series.Series_prefix.dropna()):
    entity_list = entity_name.split(',')
    for ent in entity_list:
        if ent=='US':
            var_list.append(series_name[1:])
            
var_list[-2:]

In [ ]:
path = 'Input/Oxford/2021Q3/'
os.listdir(path)

In [ ]:
def oe_to_df_A(path, vars_important_list):
    
    df = pd.read_excel(path, sheet_name='Annual', header=1)
    # Select rows
    mask = df['Indicator code'].isin(vars_important_list)
    df = df.loc[mask]
    
    # Select cols
    years = df.filter(regex=r'\d{4}').columns.values.tolist()
    other_cols = ['Indicator code']
    cols_final = other_cols+years 
    
    # Pivot table
    df = df[cols_final].pivot_table(columns='Indicator code')
    
    # Datetime change
    df.index = [(x).replace(" ",'') for x in df.index] #Removes space
    df.index = pd.PeriodIndex(pd.to_datetime(df.index),freq='A')
    return df

In [ ]:
df_oe = oe_to_df_A('Input/Oxford/2021Q3/2021_10_08_Sep21_2_WSJ_ATL_annual.xlsx',var_list)
df_oe = oe_annual.add_prefix('US_')
df_oe['2020':].head(2)

#10

In [ ]:
import sys

In [ ]:
print(sys.path)

In [ ]:
import sqlite3

In [ ]:

sys.path.append('Modules')
from Forecast_data_state_20210916 import * 

instance = merge_data_state(state = 'California', 
                            state_prefix = 'CA_')
instance.download_datasets()
instance.download_tabs(True)
df_state = instance.create_df(annual=True)
df_state = df_state['1980':]
df_state.head(2)

# acs/ cirb/ dataquick

In [ ]:
frames = [city_final,df_cirb, dq_data]
final = pd.concat(frames,axis=1)


# df_1990_f
# 'Corona', 'Lake Elsinore', 'Norco', 'Riverside'

In [ ]:
DOF_1 = pd.concat([city_final,df_cirb, dq_data, df_1990_f],axis=1)
DOF_1

# df_2000_f
# 'Eastvale', 'Jurupa Valley'

In [ ]:
DOF_2 = pd.concat([city_final,df_cirb, dq_data, df_2000_f], axis=1)

In [ ]:
DOF_2

# final merge

# 1.1 DOF_1

In [ ]:
city_list = ['Corona', 'Riverside', 'Norco', 'Eastvale', 'Jurupa Valley', 'Lake Elsinore']

In [ ]:
merged_city = pd.concat([df_oe,df_state],axis=1)
merged_city.tail(5)

In [ ]:
df_f = DOF_1.reset_index(level=0).join(merged_city) 
df_f.columns

In [ ]:
df_f.tail(3)

In [ ]:
for city in city_list:
    mask = (df_f['level_0']==city)
    merged_city = df_f.loc[mask,:]
    merged_city.to_csv('DOF_1_model_data' + city +'.csv')

# 1.2 DOF_2

In [ ]:
d_f = DOF_2.reset_index(level=0).join(merged_city) 
d_f.columns

In [ ]:
d_f.tail(3)

In [ ]:
for city in city_list:
    mask = (d_f['level_0']==city)
    merged_city = d_f.loc[mask,:]
    merged_city.to_csv('DOF_2_model_data' + city +'.csv')